This notebook takes a look at the results from a hypothetical slot machine through the lens of probability
* Tutors:
  * Anthropic's AI Claude
  * Google's Gemini

Given a slot machine and five observed payouts

Observed numbers = [0,0,1.5,0,1.5]
  * Sum = 3
  * Average = 3/5 = 0.6
  * Sample size = 5



Methods used for developing probability estimates
* Guess 1 = continous probability distribution, normal distribution
* Guess 2 = discrete probability distribution, binomal distribution
* Guess 3 = bayseian approach
* Guess 4 = monte carlo approach

In [18]:
#Given a slot machine and five observed payouts

#Observed numbers = [0,0,1.5,0,1.5]
    #Sum = 3
    #Average = 3/5 = 0.6
    #Sample size = 5

    #Guess 1 = continous probability distribution, normal distribution
    #Guess 2 = discrete probability distribution, binomal distribution
    #Guess 3 = bayseian approach to a probability estimate
    #Guess 4 = monte carlo approach to a probability estimate

import numpy as np
from scipy.stats import binom

def calc_mean(numbers):
    '''
    Manual mean calculator
    '''

    total = sum(numbers)
    pop = len(numbers)

    mean = total/pop

    return mean

def calc_median(numbers):
    """
    Manual median calculator

    Tutor: Anthropic's AI Claude
    """

    sorted_numbers = sorted(numbers)
    n = len(sorted_numbers)

    if n % 2 == 0:
        # If even length, average the two middle numbers
        return (sorted_numbers[n//2 - 1] + sorted_numbers[n//2]) / 2
    else:
        # If odd length, return the middle number
        return sorted_numbers[n//2]

def calc_standard_deviation(numbers):
    """
    Manual Standard Deviation Calculator

    Calculates the spread of the slot machine payouts

    Tutor: Anthropic's AI Claude
    """
    n = len(numbers)

    # Calculate mean
    mean = sum(numbers) / n

    # Calculate squared differences from mean
    squared_diff_sum = sum((x - mean) ** 2 for x in numbers)

    # Calculate variance and standard deviation
    variance = squared_diff_sum / n
    std_dev = variance ** 0.5

    return std_dev

#import numpy as np

def normal_distribution(x, mu, sigma):
    """
    Calculates the normal distribution probability density at point x

    The equation gives the probability density at any point x for a distribution with specified mean and standard deviation.

    When we calculate the normal distribution probability density at the mean (x = 0.6), we get approximately 0.4843. This value represents
    the relative likelihood of observing a payout around 0.6 units if the payouts truly follow a normal distribution.

    Uses the numpy library

    Parameters:
    x (float): Point at which to evaluate the distribution
    mu (float): Mean of the distribution
    sigma (float): Standard deviation of the distribution

    Returns:
    float: Probability density at point x

    Tutor: Anthropic's AI Claude

    """
    nd = (1 / (sigma * np.sqrt(2 * np.pi))) * np.exp(-0.5 * ((x - mu) / sigma)**2)

    return nd

#import numpy as np
#from scipy.stats import binom

def binomial_distribution(k, n, p):
    """
    Calculates the binomial probability mass function at point k
    The equation gives the probability of exactly k successes in n trials,
    where each trial has probability p of success.

    For the slot machine example with [0, 0, 1.5, 0, 1.5]:
    - n = 5 (total spins)
    - k = 2 (number of 1.5 payouts)
    - p = 0.4 (probability of getting 1.5, which occurred 2/5 times)
    Using these values would give us the probability of getting exactly 2 payouts of 1.5 in 5 spins.
    Uses the scipy.stats library for accurate calculation

    Parameters:
    k (int): Number of successes to calculate probability for
    n (int): Total number of trials
    p (float): Probability of success on each trial (between 0 and 1)

    Returns:
    float: Probability mass for exactly k successes

    Uses the numpy and scipy.stats libraries for accurate calculation

    Tutor: Anthropic's AI Claude

    Examples:
    >>> binomial_distribution(k=2, n=5, p=0.4)  # Probability of 2 successes in 5 trials with 0.4 probability each
    0.2304
    """
    # Using scipy's binom.pmf for accurate calculation
    # Alternative manual calculation would be:
    # probability = (np.math.comb(n, k)) * (p**k) * ((1-p)**(n-k))
    probability = binom.pmf(k, n, p)
    return probability




def bayesian_slot_probability(observed_data, prior_prob=0.4):
    """
    Bayesian approach to calculating the probability of winning on a slot machine
    based on observed data.

    Parameters:
    observed_data (list): List of observed payouts (e.g., [0, 0, 1.5, 0, 1.5])
    prior_prob (float): Prior probability of winning, defaults to 0.4 based on observed frequency

    Returns:
    float: Posterior probability of winning on the next spin

    The function uses Bayes' theorem:
    P(Win|Data) = P(Data|Win) * P(Win) / P(Data)

    For slot machines:
    - P(Win|Data) is our updated probability of winning after seeing the data
    - P(Data|Win) is probability of seeing our specific sequence given win probability
    - P(Win) is our prior belief about win probability (0.4 default)
    - P(Data) is probability of observing our sequence under any probability



    # Count wins (payouts > 0) and total observations
    wins = sum(1 for payout in observed_data if payout > 0)
    total_spins = len(observed_data)

    # Calculate likelihood: P(Data|Win)
    # This is the probability of observing this many wins given our prior
    likelihood = (prior_prob ** wins) * ((1 - prior_prob) ** (total_spins - wins))

    # Calculate marginal probability: P(Data)
    # For simplification, we can use the observed frequency
    marginal = wins / total_spins

    # Calculate posterior probability using Bayes' theorem
    posterior_prob = (likelihood * prior_prob) / marginal if marginal > 0 else prior_prob

    # Ensure probability is between 0 and 1
    posterior_prob = min(1.0, max(0.0, posterior_prob))


    P(A|B) = (P(B|A) * P(A)) / P(B)

    P(A|B) = Posterior probability
    P(B|A) = Likelihood
    P(A) = Prior probability
    P(B) = Marginal probability

    Tutors:
      Google's AI Gemini
      Anthropic's AI Claude

    """
    # Count wins (payouts > 0) and total observations
    wins = sum(1 for payout in observed_data if payout > 0)
    total_spins = len(observed_data)

    # Use the Beta distribution conjugate prior
    # Beta(α + wins, β + losses) where α=β=1 for uniform prior
    alpha = 1 + wins
    beta = 1 + (total_spins - wins)

    # Posterior probability is the expected value of the Beta distribution
    posterior_prob = alpha / (alpha + beta)

    return posterior_prob


def monte_carlo():
    """
    Monte Carlo approach to calculating probability

    E(x) = Σ(x_i * p_i) / Σ(p_i)

    E(x) = Expected value
    x_i = Individual values
    p_i = Probabilities for each value

    Tutors:
      Google's AI Gemini
      Anthropic's AI Claude

    """
    E = 0
    for i in range(len(numbers)):
        E += numbers[i] * p[i]

    return E


def estimate_average_slot_payout(n_runs):
    """Run the slot machine n_runs times and return the average net profit per run.

    Example calls (note that return value is nondeterministic!):

    Tutors:
      Google's Gemini
      Anthropic's AI Claude

    """
    numbers = [0,0,1.5,0,1.5]

    sum = 0
    for i in range(n_runs):
        sum += sum(numbers)

    total_profit = 0
    for i in range(n_runs):
        total_profit += sum(numbers)

    pop = len(numbers)

    total_profit = sum(numbers)/pop
    average_profit = total_profit / n_runs

    return average_profit

# Observed results
numbers = [0,0,1.5,0,1.5]
print("Observed results:", numbers, "\n")
print("Sample size:", len(numbers), "\n")
print(f"Observed wins: {sum(1 for x in numbers if x > 0)} out of {len(numbers)} spins\n")

#print("Mean is ", calc_mean(numbers))
mean = calc_mean(numbers)
print(f"Mean: {mean}\n")

#print("Median is ", calc_median(numbers))
median = calc_median(numbers)
print(f"Median: {median}\n")

#print("Standard deviation is ", calc_standard_deviation(numbers))
StanDev = calc_standard_deviation(numbers)
print(f"Standard deviation: {StanDev}\n")

# Example - binomial approach
k = 2 #number of 1.5 payouts
n = 5 #total spins
p= 0.4 #probability of getting 1.5, which occurred 2/5 times
print("Binomial distribution", binomial_distribution(k, n, p))
print("A discrete probability distribution/binomial might be more appropriate than a continuous/normal one.")
print("This is because it's similar to a 'success/failure' pattern.")
print("This value represents the relative likelihood of observing a payout around 0.6 units if the payouts truly follow a binomial distribution.\n")

# Example - normal/gaussian approach
x = mean # When we evaluate the normal distribution at x = mean (0.6), we get the peak density for your distribution.
mu = median
sigma = StanDev
print("Normal distribution:", normal_distribution(x, mu, sigma))
print("When we calculate the normal distribution probability density at the mean (x = 0.6), we get approximately 0.4843. ")
print("This value represents the relative likelihood of observing a payout around 0.6 units if the payouts truly follow a normal distribution.\n")

# Example - bayesian approach
posterior_probability = bayesian_slot_probability(numbers)
print(f"Bayesian posterior probability of winning: {posterior_probability:.4f}")

Observed results: [0, 0, 1.5, 0, 1.5] 

Sample size: 5 

Observed wins: 2 out of 5 spins

Mean: 0.6

Median: 0

Standard deviation: 0.7348469228349535

Binomial distribution 0.34559999999999974
A discrete probability distribution/binomial might be more appropriate than a continuous/normal one.
This is because it's similar to a 'success/failure' pattern.
This value represents the relative likelihood of observing a payout around 0.6 units if the payouts truly follow a binomial distribution.

Normal distribution: 0.38899888689271633
When we calculate the normal distribution probability density at the mean (x = 0.6), we get approximately 0.4843. 
This value represents the relative likelihood of observing a payout around 0.6 units if the payouts truly follow a normal distribution.

Bayesian posterior probability of winning: 0.4286
